# Decision Trees – Theoretical Explanation

---

## 1. What is a Decision Tree?

A **Decision Tree** is a supervised machine learning algorithm used for both **classification** and **regression**.  
It models decisions in the form of a **tree-like structure**, where:

- **Root node** represents the entire dataset
- **Internal nodes** represent feature-based decisions
- **Branches** represent outcomes of those decisions
- **Leaf nodes** represent final predictions (class labels or values)

Decision trees work by **recursively splitting the data** into smaller and more homogeneous subsets.

---

## 2. Structure of a Decision Tree

### 2.1 Nodes in a Tree

- **Root Node**  
  The topmost node where the first split happens.

- **Decision / Internal Nodes**  
  Nodes that split data based on a feature and threshold.

- **Leaf Nodes**  
  Terminal nodes that give the final output.

---

## 3. How Decision Trees Learn

The learning process follows a **top-down greedy approach**:

1. Choose the best feature to split the data.
2. Split the dataset into subsets.
3. Repeat recursively for each subset.
4. Stop when:
   - All samples belong to the same class, or
   - Maximum depth is reached, or
   - Minimum samples per node condition is met.

---

## 4. Splitting Criteria

To decide the best split, decision trees use **impurity measures**.

---

## 5. Gini Impurity

### 5.1 Definition

Gini impurity measures how often a randomly chosen element would be incorrectly classified.

$$
Gini = 1 - \sum_{i=1}^{C} p_i^2
$$

Where:
- $p_i$ is the probability of class $i$
- \(C\) is the number of classes

### 5.2 Interpretation

- Gini = 0 → Pure node
- Higher Gini → More mixed classes

### 5.3 Why Gini?

- Faster computation
- Commonly used in practice (e.g., CART algorithm)

---

## 6. Entropy and Information Gain

### 6.1 Entropy

Entropy measures the **amount of uncertainty** in the data.

$$
Entropy = -\sum_{i=1}^{C} p_i \log_2(p_i)
$$

- Entropy = 0 → Pure node
- Maximum when classes are equally distributed

---

### 6.2 Information Gain

Information Gain tells us how much uncertainty is reduced after a split.

$$
IG = Entropy(parent) - \sum_{j} \frac{n_j}{n} Entropy(child_{j})
$$

The feature with the **highest Information Gain** is chosen for splitting.

---

## 7. Comparison: Gini vs Entropy

| Aspect | Gini | Entropy |
|------|------|---------|
| Basis | Probability of misclassification | Information theory |
| Speed | Faster | Slower |
| Split Quality | Similar | Slightly better sometimes |
| Usage | Default in sklearn | Optional |

---

## 8. Decision Boundaries

- Decision trees create **axis-aligned splits**
- Resulting decision boundaries are **rectangular**
- More depth → more complex boundaries

---

## 9. Overfitting in Decision Trees

### Why Overfitting Happens

- Trees can grow very deep
- Can perfectly memorize training data
- Sensitive to noise

### Symptoms

- Very high training accuracy
- Lower test accuracy

---

## 10. Controlling Overfitting

### 10.1 Pre-Pruning (Early Stopping)

- `max_depth`
- `min_samples_split`
- `min_samples_leaf`
- `max_features`

### 10.2 Post-Pruning

- Remove branches after training
- Cost-complexity pruning (advanced)

---

## 11. Feature Importance in Decision Trees

Decision trees naturally compute feature importance based on:

- Reduction in impurity
- Frequency of feature usage

Higher importance → feature contributes more to predictions.

---

## 12. Advantages of Decision Trees

- Easy to interpret and visualize
- Handles non-linear relationships
- No need for feature scaling
- Works with numerical and categorical data

---

## 13. Limitations of Decision Trees

- Prone to overfitting
- Unstable to small data changes
- Axis-aligned splits only
- Lower accuracy compared to ensembles

---

## 14. Decision Trees vs Other Models

| Model | Interpretability | Non-linearity | Scaling Needed |
|-----|------------------|--------------|---------------|
| Decision Tree | High | Yes | No |
| Logistic Regression | Medium | No | Yes |
| KNN | Low | Yes | Yes |
| SVM | Low | Yes | Yes |

---

## 15. Real-World Applications

- Medical diagnosis
- Credit approval
- Customer churn prediction
- Risk assessment
- Rule-based decision systems

---

## 16. Key Takeaways

- Decision trees split data using impurity measures
- Gini and Entropy are most common criteria
- Trees can easily overfit if unrestricted
- Limiting depth improves generalization
- Visualization helps interpret decisions

---

## 17. What to Learn Next

- Random Forests
- Gradient Boosting Trees
- XGBoost / LightGBM
- Tree pruning techniques

---


### 🌳 Decision Tree Classifier (From Scratch)

In [13]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# 1. Load data
data = load_iris()
X, y = data.data, data.target
feature_names = data.feature_names
class_names = data.target_names

# 2. CORE MATHEMATICS: GINI IMPURITY
def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return 1 - np.sum(probs**2)

# 3. SPLIT LOGIC
def split_dataset(X, y, feature, threshold):
    left_mask = X[:, feature] <= threshold
    right_mask = X[:, feature] > threshold
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

def best_split(X, y):
    best_gini = float("inf")
    best_feature = None
    best_threshold = None
    
    n_features = X.shape[1]
    
    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            # Weighted Gini Impurity
            w_left = len(y_left) / len(y)
            w_right = len(y_right) / len(y)
            current_gini = (w_left * gini_impurity(y_left)) + (w_right * gini_impurity(y_right))
            
            if current_gini < best_gini:
                best_gini = current_gini
                best_feature = feature
                best_threshold = threshold
                
    # FIX: Return outside the loops so we check ALL features/thresholds
    return best_feature, best_threshold

# 4. TREE ARCHITECTURE
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # If value is not None, this is a Leaf Node

def build_tree(X, y, depth=0, max_depth=3):
    n_samples, n_features = X.shape
    unique_classes = np.unique(y)
    
    # Stopping Criteria: Pure node, Max depth, or no samples
    if len(unique_classes) == 1 or depth == max_depth or n_samples < 2:
        return TreeNode(value=np.bincount(y).argmax())
    
    feature, threshold = best_split(X, y)
    
    # FIX: Correct syntax for None check
    if feature is None:
        return TreeNode(value=np.bincount(y).argmax())
    
    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
    
    # Recursive builds
    left_child = build_tree(X_left, y_left, depth + 1, max_depth)
    right_child = build_tree(X_right, y_right, depth + 1, max_depth)
    
    return TreeNode(feature, threshold, left_child, right_child)

# 5. PREDICTION
def predict_sample(node, x):
    if node.value is not None:
        return node.value
    
    if x[node.feature] <= node.threshold:
        return predict_sample(node.left, x)
    return predict_sample(node.right, x)

def predict(tree, X):
    return np.array([predict_sample(tree, x) for x in X])

# 6. RUN AND PRINT
tree = build_tree(X, y, max_depth=3)
preds = predict(tree, X)
print(f"Scratch Tree Accuracy: {np.mean(preds == y) * 100:.2f}%")

def print_tree(node, depth=0):
    if node.value is not None:
        print("  " * depth + f"Leaf → Class {class_names[node.value]}")
        return

    print("  " * depth + f"[{feature_names[node.feature]} ≤ {node.threshold:.2f}]")
    print_tree(node.left, depth + 1)
    print_tree(node.right, depth + 1)

print("\n--- Tree Structure ---")
print_tree(tree)

Scratch Tree Accuracy: 97.33%

--- Tree Structure ---
[petal length (cm) ≤ 1.90]
  Leaf → Class setosa
  [petal width (cm) ≤ 1.70]
    [petal length (cm) ≤ 4.90]
      Leaf → Class versicolor
      Leaf → Class virginica
    [petal length (cm) ≤ 4.80]
      Leaf → Class virginica
      Leaf → Class virginica


--------
--------

#### 📘 How Decision Trees Work (Jupyter Markdown)
1. Gini Impurity: Measuring "Messiness"

The Decision Tree wants to split the data so that the resulting groups are as "pure" as possible. We use Gini Impurity to measure this.

The Mathematical Formula:
$$Gini = 1 - \sum_{i=1}^{C} p_i^2$$

- If a node contains only one class, $Gini = 0$ (Perfectly pure).
- If a node is a 50/50 mix of two classes, $Gini = 0.5$ (Maximum impurity).

2. Recursive Splitting (Divide & Conquer)

The build_tree function uses recursion. It behaves like a greedy algorithm:
- At every node, it looks at every feature and every possible value to split on.
- It chooses the split that reduces the total Gini impurity the most.
- It repeats this process for the "left" and "right" subsets until it hits a Stopping Criterion.

Common Stopping Criteria:

- Max Depth: Prevents the tree from becoming too complex (overfitting).
- Pure Node: All samples in the node belong to the same class.
- Minimum Samples: Don't split if a node has too few data points.

3. Decisions: Thresholds

Unlike KNN (which uses distances) or Logistic Regression (which uses weights), a Decision Tree uses Thresholds. It creates axis-aligned boundaries in the feature space.

- Example: If Petal Width <= 0.8, the Iris is almost certainly a Setosa.
- The model doesn't care about the actual value of the Petal Width, only whether it is on the "left" or "right" of that specific number.